In [ ]:
import sys
import glob
import re
import fnmatch
import math
import os
from os import listdir
from os.path import join, isfile, basename
import itertools

import numpy as np
from numpy import float32, int32, uint8, dtype, genfromtxt

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

import scipy
import pandas as pd
import colorsys

import template_common as tc

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
base_dir='/nrs/saalfeld/john/projects/flyChemStainAtlas/all_evals/distanceStatsWarpNorm'

dest_dir = '/groups/saalfeld/home/bogovicj/pubDrafts/grpDrosTemplate/grpDrosTemplate/tables'
fig_dir = '/groups/saalfeld/home/bogovicj/pubDrafts/grpDrosTemplate/grpDrosTemplate/figs'


alg_list = ['antsRegDog', 'antsRegOwl', 'antsRegYang', 'cmtkCOG', 'cmtkCow', 'cmtkHideo']
# template_list = [ 'JFRC2013_lo', 'JFRCtemplate2010', 'TeforBrain_f', 'F-antsFlip_lo', 'F-cmtkFlip_lof', 'FCWB']
template_list = [ 'JFRC2013_lo', 'JFRCtemplate2010', 'TeforBrain_f', 'F-antsFlip_lo', 'FCWB']

pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
# Load everything
# os.listdir( base_dir )

df = None
# for tmp in template_list:
for f in glob.glob( ''.join([base_dir,'/*.csv']) ):
#     f = glob.glob( ''.join([base_dir,'/',tmp,'.csv']) )
    print( f )
    this_df = pd.read_csv( f, header=[0,1], index_col=0 )
    if df is None:
        df = this_df
    else:
        df = df.append( this_df )

tc.clean_cols( df )
df['std'] = df.apply( lambda x: math.sqrt(x['var']), axis=1)
df['gam_std'] = df.apply( lambda x: math.sqrt(x['gam_var']), axis=1)
df['ray_std'] = df.apply( lambda x: math.sqrt(x['ray_var']), axis=1)
df.reset_index( drop=True )

In [ ]:
tmask = df.apply( lambda x: (x['TEMPLATE'] in template_list ), axis=1)
df = df.loc[tmask]

df_ed = df.loc[ (df.ALG != 'ALL')]
df_ed['TEMPLATE'] = df_ed.apply(lambda x: tc.template_name(x['TEMPLATE']), axis=1)
df_ed['ALG'] = df_ed.apply(lambda x: tc.alg_name(x['ALG']), axis=1)
# df_ed['TA'] = df_ed.apply(lambda x: ''.join([x['TEMPLATE'],':',x['ALG']]), axis=1)

In [ ]:
# df_ed[[['mean','median','std']]]
# df_ed['ALG'].unique()

df_ed.head()

In [ ]:

# print( df_writeme.index)
# mi = pd.MultiIndex.from_product([df_l.TEMPLATE.unique(), df_l.ALG.unique()], names=['Template','Algorithm'])
# mi

# df_writeme.set_index( mi )
# df_writeme.set_index( ['TEMPLATE','ALG'])

# print( df_writeme.set_index( ['Template','Algorithm']).to_latex( multirow='True'))
# print( df_writeme.to_latex())

In [ ]:
template_order = ['JFRC2010','JFRC2013','JRC2018','FCWB','Tefor']
alg_order = ['ANTs A','ANTs B','ANTs C','CMTK A','CMTK B','CMTK C']

for label in tc.labels:
    print( label )
    df_l = df_ed[ df_ed.LABEL == label ]
    # df_writeme = df_l[['TEMPLATE','ALG','mean','std','p10','median','p90','count']].sort_values(by=['TEMPLATE','ALG'])
    df_writeme = df_l[['TEMPLATE','ALG','mean','std','p10','median','p90','count']]
    df_writeme.columns = ['Template','Algorithm','Mean','Std dev','10th perc','median','90th perc','N']
    df_writeme = df_writeme.set_index( ['Template','Algorithm'])

    # df_writeme

    # df_writeme.reindex(labels=template_order, axis='index')
    # df_writeme.loc[['JFRC2010','JFRC2013','JRC2018''FCWB','Tefor']]
    #     print(df_writeme)

#     print( '\caption{{ {} }}'.format( tc.get_label_string( label ).replace('_','\_')) )
    
    f = '/nrs/saalfeld/john/projects/flyChemStainAtlas/all_evals/distanceStatsWarpNorm/tables/label_{}.tex'.format(label)
    with open( f, 'w') as tex_file:
        print( '\\begin{table}', file=tex_file )
        print( df_writeme.to_latex( multirow='True'), file=tex_file)
        print( '\caption{{ {} }}'.format( tc.get_label_string( label ).replace('_','\_')), file=tex_file )
        print( '\caption{{ {} }}'.format( tc.get_label_string( label ).replace('_','\_')) )
        print( '\end{table}', file=tex_file )

# print( ' ' )
# break